In [ ]:
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.9 MB/s eta 0:00:00


In [ ]:
pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 20.0 MB/s eta 0:00:00


In [ ]:
!pip install --user easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 106.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 35.0 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
# ================================
# REQUIRED IMPORTS
# ================================
import os
from glob import glob
from xml.dom import minidom

# ================================
# CLASS LABEL LOOKUP TABLE
# (ONLY ONE CLASS: LICENSE PLATE)
# ================================
lut = {
    "licence_plate": 0,
    "license_plate": 0
}

# ================================
# FUNCTION: CONVERT XML → YOLO
# ================================
def convert_xml2yolo(lut, input_path, output_path):

    # Create output directory (including parent folders)
    os.makedirs(output_path, exist_ok=True)

    xml_files = glob(os.path.join(input_path, "*.xml"))
    print(f"Found {len(xml_files)} XML files")

    for fname in xml_files:

        # Parse XML
        xmldoc = minidom.parse(fname)

        # Output label filename
        image_name = os.path.splitext(os.path.basename(fname))[0]
        label_file = os.path.join(output_path, image_name + ".txt")

        with open(label_file, "w") as f:

            # Image size
            size = xmldoc.getElementsByTagName("size")[0]
            width = int(size.getElementsByTagName("width")[0].firstChild.data)
            height = int(size.getElementsByTagName("height")[0].firstChild.data)

            # Objects in image
            objects = xmldoc.getElementsByTagName("object")

            for obj in objects:
                class_name = obj.getElementsByTagName("name")[0].firstChild.data.strip()

                # Get class id (default 0 if unknown)
                class_id = lut.get(class_name, 0)

                bndbox = obj.getElementsByTagName("bndbox")[0]
                xmin = float(bndbox.getElementsByTagName("xmin")[0].firstChild.data)
                ymin = float(bndbox.getElementsByTagName("ymin")[0].firstChild.data)
                xmax = float(bndbox.getElementsByTagName("xmax")[0].firstChild.data)
                ymax = float(bndbox.getElementsByTagName("ymax")[0].firstChild.data)

                # Convert to YOLO format (normalized)
                x_center = ((xmin + xmax) / 2.0) / width
                y_center = ((ymin + ymax) / 2.0) / height
                w = (xmax - xmin) / width
                h = (ymax - ymin) / height

                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

    print("✅ XML → YOLO conversion completed successfully!")

# ================================
# PATHS (YOUR CURRENT STRUCTURE)
# ================================
input_path = "/content/drive/MyDrive/archive (1)/annotations"
output_path = "/content/drive/MyDrive/output_yolov8_numberplate/annotaions"

# ================================
# RUN CONVERSION
# ================================
convert_xml2yolo(lut, input_path, output_path)


Found 433 XML files
✅ XML → YOLO conversion completed successfully!


In [ ]:
def create_directories_if_not_exist(directories: list):
    # Assert: Is the parameter a list?
    assert isinstance(directories, list), "The parameter must be a list (array)."

    # Assert: Is every element in the list a string?
    for dir_path in directories:
        assert isinstance(dir_path, str), f"The element '{dir_path}' in the list is not a string."

    # Create directories
    for dir_path in directories:
        if not os.path.exists(dir_path):
            os.mkdir(dir_path)
            print(f"Created: {dir_path}")
        else:
            print(f"Already exists: {dir_path}")


In [ ]:
directories = [
    "/content/drive/MyDrive/archive (1)/annotations",
    "/content/drive/MyDrive/archive (1)/annotations/images",
    "/content/drive/MyDrive/archive (1)/annotations/labels",
    "/content/drive/MyDrive/archive (1)/annotations/images/train",
    "/content/drive/MyDrive/archive (1)/annotations/images/val",
    "/content/drive/MyDrive/archive (1)/annotations/labels/train",
    "/content/drive/MyDrive/archive (1)/annotations/labels/val"
]

create_directories_if_not_exist(directories)

Already exists: /content/drive/MyDrive/archive (1)/annotations
Already exists: /content/drive/MyDrive/archive (1)/annotations/images
Already exists: /content/drive/MyDrive/archive (1)/annotations/labels
Already exists: /content/drive/MyDrive/archive (1)/annotations/images/train
Already exists: /content/drive/MyDrive/archive (1)/annotations/images/val
Already exists: /content/drive/MyDrive/archive (1)/annotations/labels/train
Already exists: /content/drive/MyDrive/archive (1)/annotations/labels/val


In [ ]:
def split_images_and_labels(val_size = 0.1, input_dir = "/content/drive/MyDrive/archive (1)",
                           output_dir = "/content/drive/MyDrive/output_yolov8_numberplate"):

        img_path = os.listdir(f"{input_dir}/images")
        label_path = os.listdir(f"{input_dir}/annotations")
        imgs_length = len(img_path)
        for i,img in enumerate(img_path):
            spname = "train" if i < int(imgs_length*float(1 - val_size)) else "val"

            #print(spname)

            os.system(f"cp {input_dir}/images/{img} {output_dir}/images/{spname}/{img}")
            #print(f"the image {input_dir}/images/{img} image copied to {output_dir}/images/{spname}/{img}")

            os.system(f"cp /content/drive/MyDrive/output_yolov8_numberplate/{img.split('.')[0]}.txt {output_dir}/labels/{spname}/{img.split('.')[0]}.txt")
            #print(f"the label /kaggle/working/annotations/{img.split('.')[0]}.txt image copied to {output_dir}/labels/{spname}/{img.split('.')[0]}.txt")

        print("Splitting is finished!")

split_images_and_labels()


Splitting is finished!


In [ ]:
!pip install pyyaml


In [ ]:
import yaml


In [ ]:
data = {
    'train': '/content/drive/MyDrive/archive (1)/annotations/images/train',
    'val': '/content/drive/MyDrive/archive (1)/annotations/images/val',
    'names': {
        0: "number_plate"
    } # List formatında olmalı!
}

# Writing the data to a YAML file
with open('data.yaml', 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

print("Data has been written to 'data.yaml'")

Data has been written to 'data.yaml'


In [ ]:
import os
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# -----------------------------
# CORRECT PATHS
# -----------------------------
image_dir = "/content/drive/MyDrive/archive (1)/images"
label_dir = "/content/drive/MyDrive/output_yolov8_numberplate/annotaions"

image_extensions = ["*.jpg", "*.jpeg", "*.png"]

# Collect image paths
image_paths = []
for ext in image_extensions:
    image_paths.extend(glob.glob(os.path.join(image_dir, ext)))

print("Images found:", len(image_paths))
print("Sample images:", image_paths[:3])

image_paths = sorted(image_paths)[:9]

if len(image_paths) == 0:
    print("❌ No images found. Check image_dir path.")

fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes = axes.flatten()

for idx, image_path in enumerate(image_paths):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w, _ = img.shape

    base_name = os.path.splitext(os.path.basename(image_path))[0]
    label_path = os.path.join(label_dir, base_name + ".txt")

    ax = axes[idx]
    ax.imshow(img)
    ax.axis("off")

    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            lines = f.readlines()

        for line in lines:
            class_id, x_center, y_center, bw, bh = map(float, line.split())

            x1 = int((x_center - bw / 2) * w)
            y1 = int((y_center - bh / 2) * h)
            x2 = int((x_center + bw / 2) * w)
            y2 = int((y_center + bh / 2) * h)

            rect = patches.Rectangle(
                (x1, y1),
                x2 - x1,
                y2 - y1,
                linewidth=2,
                edgecolor="red",
                facecolor="none"
            )
            ax.add_patch(rect)
    else:
        print("⚠️ Label missing for:", base_name)

plt.tight_layout()
plt.show()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
data_yaml = """
path:  /content/drive/MyDrive
train: archive (1)/images
val: archive (1)/images

names:
  0: license_plate
"""

with open("data.yaml", "w") as f:
    f.write(data_yaml)

print("✅ data.yaml created")

✅ data.yaml created


In [ ]:
!pip install torch torchvision torchaudio


In [ ]:
import os, shutil

src = "/content/drive/MyDrive/output_yolov8_numberplate/annotaions"
dst = "/content/drive/MyDrive/archive (1)/labels"

os.makedirs(dst, exist_ok=True)

for f in os.listdir(src):
    shutil.move(os.path.join(src, f), os.path.join(dst, f))

print("✅ Labels moved")


✅ Labels moved


In [ ]:
images = set(os.path.splitext(f)[0] for f in os.listdir("/content/drive/MyDrive/archive (1)/images"))
labels = set(os.path.splitext(f)[0] for f in os.listdir("/content/drive/MyDrive/archive (1)/labels"))

print("Images without labels:", list(images - labels)[:10])
print("Labels without images:", list(labels - images)[:10])

Images without labels: []
Labels without images: []


In [ ]:
with open("/content/drive/MyDrive/archive (1)/labels/Cars0.txt") as f:
    print(f.read())


0 0.645000 0.555970 0.386000 0.179104



In [ ]:
import torch


In [ ]:
from ultralytics import YOLO


In [96]:
# Check the number of GPUs
gpu_count = torch.cuda.device_count()
device = list(range(gpu_count)) if gpu_count > 1 else 0

model = YOLO("yolo11n.pt")

results = model.train(
    data="data.yaml",   # Dataset configuration
    epochs=100,                         # 200 epochs
    imgsz=640,                          # Suitable for smaller images
    batch=32,                            # Adjustable based on GPU RAM
    workers=2,                          # Ideal starting value for Tesla T4
    device=device,                      # GPU setting

    # ✅ Default Regularization & Optimization (YOLO uses its own defaults)
    # If you don't define these parameters, defaults will be used.

    # ✅ Default Augmentations
    augment=True,                       # Default YOLO augmentations are automatically enabled

    # ✅ Early Stopping and Validation
    patience=20,                        # Stops early if no improvement for 10 epochs
    val=True,                           # Validation is performed at the end of each epoch
)

Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train5, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0, plots=True, pose=12.0, pretrained=True, 

In [97]:
!zip -r train5_results.zip runs/detect/train5


  adding: runs/detect/train5/ (stored 0%)
  adding: runs/detect/train5/train_batch1261.jpg (deflated 19%)
  adding: runs/detect/train5/val_batch2_labels.jpg (deflated 11%)
  adding: runs/detect/train5/val_batch0_pred.jpg (deflated 12%)
  adding: runs/detect/train5/train_batch1.jpg (deflated 4%)
  adding: runs/detect/train5/results.png (deflated 8%)
  adding: runs/detect/train5/val_batch0_labels.jpg (deflated 12%)
  adding: runs/detect/train5/train_batch1262.jpg (deflated 12%)
  adding: runs/detect/train5/BoxF1_curve.png (deflated 15%)
  adding: runs/detect/train5/weights/ (stored 0%)
  adding: runs/detect/train5/weights/best.pt (deflated 10%)
  adding: runs/detect/train5/weights/last.pt (deflated 10%)
  adding: runs/detect/train5/val_batch2_pred.jpg (deflated 11%)
  adding: runs/detect/train5/train_batch0.jpg (deflated 6%)
  adding: runs/detect/train5/train_batch1260.jpg (deflated 15%)
  adding: runs/detect/train5/confusion_matrix_normalized.png (deflated 36%)
  adding: runs/detect/tra

In [98]:
from google.colab import files
files.download("train5_results.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [100]:
!mkdir -p /content/drive/MyDrive/YOLO_Models


In [101]:
!cp runs/detect/train5/weights/best.pt /content/drive/MyDrive/YOLO_Models/
